# FIA-tools

FIA-tools quantifies and measures the area of double-strand break (DSB) foci in multi-level 3D confocal images of cancer-associated fibroblasts. The program can be adapted to analyze any foci/specks detected by immunofluorescence (IF) staining, utilizing a nuclei mask for segmentation.


The main input for all of the programms is `input_paths.json`. Therefore, previously it have to be modified by a user. This file should contain a list of folders with `.nd2` images. The file can contain as many folders as needed.

In the example, you can copy the following structure in `input_paths.json`. It can be done manually in Google Colab:

```json
{
  "paths_to_files": [
    "/data/example.nd2"
  ]
}
```

Also, before starting the program, please make sure you know how many fluorescence channels you have (e.g., DAPI, Cy5) and their order in the file. You can check this by opening the image using the standard method in the GPU application [FiJi](https://imagej.net/software/fiji/downloads)

## 1_select_channels.py

This script processes confocal image stacks - `.nd2` files with Z-stacks - to analyze nuclei and foci signal channels. It is designed explicitly for immunofluorescence (IF) experiments, allowing users to extract and project specific image channels from raw .nd2 files. The results are saved as processed images in newly created directories. The script leverages ImageJ with Bio-Formats for image manipulation and Fiji plugins for Z-projection and resizing.

### How It Works
1. *Input*: The user provides a JSON file containing paths to directories with .nd2 image stacks.

2. *Channel Selection*: The script prompts users to specify channel numbers for nuclei and foci staining.

3. *Image Processing*:
  - Extracts specified channels for nuclei and foci.
  - Performs Z-projection:
    - Maximum Intensity (Max) for nuclei channel:
    - Standard Deviation (SD) for foci channel.
  - Resizes images to standard dimensions (X:1024; Y:1024) and converts them to 8-bit grayscale.

4. *Output*: Process results are saved in a folder named foci_assay, created within each input directory, and organized into subfolders:
  - `Nuclei`,
  - `Foci`

### The example of usage

1. Make the script executable

```bash
chmod +x code/1_select_channels.py
```

2. Run command

```bash
code/1_select_channels.py -i input_paths.json
```


## 2_analyse_nuclei.py

This script processes nuclei images using a machine learning-based segmentation approach powered by the pre-trained [StarDist](https://github.com/stardist/stardist) model. It is designed explicitly for high-precision nuclei segmentation in 2D fluorescent microscopy images. The output is a new set of processed masks highlighting detected nuclei, saved in well-organized directories for downstream analysis.

### How It Works

1. *Input*: The script takes a `input_paths.lson` file listing directories containing .tif images of nuclei.

  - Reads .tif images from specified directories.

  - Skips files that are not 8-bit grayscale.

2. *Normalization*:

  - Applies contrast/intensity normalization for further analysis

3. *Segmentation*: The StarDist model predicts labeled instances of nuclei with adjustable thresholds:

  - nms_thresh=0.9: Controls how close detected objects can be before merging.

  - prob_thresh=0.7: Minimum confidence required for nuclei detection.

4. *Output*: Processed masks are saved as new .tif files in a folder named *Nuclei_StarDist_mask_processed_* created for each input directory with the naming convention **_StarDist_processed*.

### The example of usage

1. Make the script executable

```bash
chmod +x code/2_analyze_nuclei.py
```

2. Run command

```bash
code/2_analyze_nuclei.py -i input_paths.json
```

## 3_filter_imgs.py

This script is designed to create binary masks for nuclei and foci in microscopy images, enabling the downstream analysis of the size and area of foci within the nuclei region. It follows the previous step, where the StarDist standard model was used to predict nuclei for a wide range of object sizes. After generating the binary masks, this script provides functionality to optimize the size threshold for nuclei objects, ensuring greater accuracy in subsequent analyses. Additionally, it converts images to binary format (black and white masks) to facilitate clear object segmentation and measurement.

### How It Works

1. *Input*: Takes directories containing the previous step's results (nuclei segmentation from StarDist). Requires user-specified thresholds for nuclei object size and foci intensity.

2. *Processing*:

  - Nuclei Masks:
    - Applies size-based thresholds to filter out unwanted nuclei objects.
    - Converts filtered nuclei masks into binary format.

  - Foci Masks:
    - Applies intensity thresholds to identify foci regions.
    - Additional watershed segmentation to separate overlapping objects.
    - Converts foci masks into binary format.
    - Associates foci with nuclei regions for future spatial analysis.
3. *Output*: Saves processed masks in separate directories for
  - nuclei: *Final_Nuclei_Mask* and
  - foci: *Foci_Mask*,
  
  with results timestamped for traceability.


### The example of usage

1. Make the script executable

```bash
chmod +x code/3_filter_imgs.py
```

2. Run command

```bash
code/3_filter_imgs.py -i input_paths.json
```

To customize the thresholds for nuclei and foci please use other arguments

```bash
code/3_filter_imgs.py -i input_paths.json -p 2000 -f 100
```

## 4_calculate_nuc_foci.py

This script represents the final step in a microscopy image analysis pipeline. It performs detailed quantification of nuclei and foci and computes summaries for detected objects' size, count, and total area. It uses binary masks from the previous steps and combines foci and nuclei masks to analyze the spatial distribution of foci within the nuclei region. Results are saved in detailed summaries for individual objects (single nuclei/foci) and aggregated data per picture (a summary of objects per folder).

### How It Works

1. *Input*: Requires a `input_paths.json` file listing paths to directories containing:
  - Binary nuclei masks: *Final_Nuclei_Mask*.
  - Binary foci masks: *Foci_Mask*.
  - Masks generated from previous pipeline steps are analyzed together.
2. *Processing*:
  - Nuclei Analysis:
    - Measures object count, total area, and average size for detected nuclei.
    - Output a summary file for each image and a combined summary for all images.
  - Foci Analysis:
    - Combines foci and nuclei masks using a logical AND operation to isolate foci within nuclei.
    - Analyzes the size and spatial distribution of foci within nuclei regions.
    - Outputs detailed and aggregated summaries.
3. *Output*:

  - Creates new folders for: Combined Masks with pictures: *Foci_in_nuclei_final_*.

  - Creates new folder foci_analysis with:
Nuclei Analysis folder: *Nuclei_count_results_*.

  - The main Analysis folder: *Foci_count_results_*. This folder is saved in dir that were mentioned in `input_paths.json`. Both folders in it contain *<folder_name>Combined_Summary.csv and processed<image_name>.csv* files.
  
    - The first file type: Each row contains information about the number of objects (nuclei or foci) detected in the image and the total area occupied by them.
    - The second file type: A separate file is generated for each image, where each row provides information about a single object, including its area (Area).

  ### The example of usage

1. Make the script executable

```bash
chmod +x code/4_calculate_nuc_foci.py
```

2. Run command

```bash
code/4_calculate_nuc_foci.py  -i input_paths.json
```

